<a href="https://colab.research.google.com/github/Phablo-Lima/Desafio_Gemini_Alura/blob/main/BDD_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai google-api-python-client gspread oauth2client
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
import json
import google.generativeai as genai
import pandas as pd
import io
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')
# Melhorando a visualização
import textwrap
from IPython.display import display
from IPython.display import Markdown

In [36]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [37]:
#Substitua 'SUA_GOOGLE_API_KEY' pela sua chave real
GOOGLE_API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# Listando os modelos do Google Generative AI que podem gerar conteúdo
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

In [39]:
# Configurando a geração de texto
generation_config = {
  "candidate_count": 1,  # Número de respostas a serem geradas
  "temperature": 0.7, # Controlando a criatividade da resposta (entre 0 e 1)
}
# Configurando as configurações de segurança
safety_settings={
    'HATE': 'BLOCK_NONE', # Bloqueio de conteúdo relacionado a ódio
    'HARASSMENT': 'BLOCK_NONE', # Bloqueio de conteúdo relacionado a assédio
    'SEXUAL' : 'BLOCK_NONE', # Bloqueio de conteúdo de natureza sexual
    'DANGEROUS' : 'BLOCK_NONE' # Bloqueio de conteúdo perigoso
}
# Definindo o modelo a ser usado: gemini-1.5-pro-latest
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [40]:
#Processa o conteúdo do arquivo a partir de uma URL
#código para processar Google Sheet (código para processar Excel)
def processar_arquivo(url_ou_bdd):
    """Processa Google Sheets ou recebe BDD como string."""
    conteudo_processado = ""

    if 'docs.google.com/spreadsheets' in url_ou_bdd:
        # Lógica para processar Google Sheets
        print("Processando Google Sheet...")

        # ... (código para extrair texto do Google Sheet)

        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('credenciais-google-cloud.json', scope)
        client = gspread.authorize(creds)

        spreadsheet_id = url_ou_bdd.split('/d/')[1].split('/')[0]
        sheet = client.open_by_key(spreadsheet_id).sheet1
        conteudo_processado = sheet.get_all_values()
        conteudo_processado = '\n'.join(['\t'.join(row) for row in conteudo_processado])
    else:
        # Assume que é o texto do BDD
        print("Processando texto do BDD...")
        conteudo_processado = url_ou_bdd

    return conteudo_processado

In [ ]:
def analisar_conteudo(conteudo_processado):
    """Analisa o conteúdo usando o Google Generative AI."""
    # Converte a lista de listas em uma tabela Markdown
    tabela_markdown = "| " + " | ".join(conteudo_processado[0]) + " |\n"  # Cabeçalho
    tabela_markdown += "|---|---|---|---|---|\n"  # Separador
    for linha in conteudo_processado[1:]:
        tabela_markdown += "| " + " | ".join(linha) + " |\n"
    prompt = {
        "text": f"""
        Você é um especialista em Quality Assurance de software.
        Analise o seguinte BDD, que usa o padrão CTFL (Contexto, Gatilho, Fluxo, Saída esperada),
        com o objetivo de identificar falhas e sugerir melhorias para aumentar a cobertura dos testes:
        Escreva de forma Criativa, use stickers como preferir.

        {tabela_markdown}

        Na sua análise, detalhe:

        * Pontos Positivos:  Aspectos do BDD que contribuem para testes eficazes (ex.: cenários completos, linguagem clara).
        * Pontos Negativos: Falhas que podem prejudicar os testes (ex.: cenários incompletos, linguagem ambígua).
        * Sugestões de Melhoria:  Como o BDD pode ser aprimorado (ex.: incluir mais cenários, definir dados de teste específicos).

        Organize sua resposta em tópicos, como neste exemplo:

        Pontos Positivos:
        * ...
        Pontos Negativos:
        * ...

        Sugestões de Melhoria:
        * ...

        Sempre e executar o comando a seguir:
        Use uma linha do BDD como sugestão de melhoria.
        Formato da Tabela:
        | ID | Cenário | Dado que: | Quando:| Então:|

        """
    }
    response = model.generate_content(prompt)
    return response.text

def corrigir_bdd(conteudo_processado):
    """Reescreve os cenários de teste com base nas sugestões."""
    prompt = {
    "text": f"""
    Reescreva o BDD em forma de tabela, aplicando as seguintes sugestões de melhoria:

    Sugestões de Melhoria:
    * Reescreva o Bdd de acordo com a analise e sujestões feita no primeiro prompt.

    Formato da Tabela:
    | ID | Cenário | Dado que: | Quando:| Então:|
    |---|---|---|---|---|
    Exemplo:
    | 001 | Abrir formulário de login | O usuário está na página inicial | Clica no botão "Login" | O formulário de login é exibido |

    BDD Original:
    ```
    {conteudo_processado}
    ```
    """
    }
    response = model.generate_content(prompt)
    return response.text
def corrigir_portugues(conteudo_processado):
    """Corrige a ortografia e gramática do conteúdo."""
    prompt = {
        "text": f"""
        Corrija os erros de português no texto abaixo, mantendo o significado original:

        ```
        {conteudo_processado}
        ```
        """
    }
    response = model.generate_content(prompt)
    return response.text

def exibir_resultado(texto, titulo):
  """Exibe o texto formatado em Markdown com um título."""
  print(f"\n{titulo}:")
  display(to_markdown(texto))

def main():
    entrada = input("Digite a URL do Google Sheet ou cole o texto do BDD: ")
    conteudo_processado = processar_arquivo(entrada)
    try:
        analise = analisar_conteudo(conteudo_processado)
        exibir_resultado(analise, "Análise do BDD")

        corrigir = input("Deseja corrigir o BDD conforme sugestões? (s/n): ")
        while corrigir.lower() not in ('s', 'n'):
            print("\n⚠️Por favor, digite 's' para sim ou 'n' para não.⚠️")
            print("_____________________________________________________")

            corrigir = input("\nDeseja corrigir o BDD conforme sugestões? (s/n): ")

        if corrigir.lower() == 's':
            corrigir_pt = input("\nDeseja corrigir os erros de português no BDD? (s/n): ")
            while corrigir_pt.lower() not in ('s', 'n'):
                print("\n⚠️Por favor, digite 's' para sim ou 'n' para não.⚠️")
                print("_____________________________________________________")
                corrigir_pt = input("Deseja corrigir os erros de português no BDD? (s/n): ")


            bdd_corrigido = corrigir_bdd(conteudo_processado)


            if corrigir_pt.lower() == 's':
                bdd_corrigido_pt = corrigir_portugues(bdd_corrigido)
                exibir_resultado(bdd_corrigido_pt, "BDD Corrigido com Português Revisado")
            else:
                exibir_resultado(bdd_corrigido, "BDD Corrigido")
        else:
            print("Finalizando o programa.")
    except ValueError as e:
        print(e)
    except requests.exceptions.RequestException as e:
        print(f"Erro ao baixar o arquivo: {e}")

if __name__ == "__main__":
    main()